In [1]:
import pandas as pd
pd.set_option("display.max_columns", None)
import re

In [2]:
#Rive Droite = 884
#Chateau = 885
#Chantiers = 886
#Porchefontaine = 679
#Montreuil = 566
stif_code = {
    "Versailles Rive Droite" : "884",
    "Versailles Château Rive Gauche" : "885",
    "Versailles Chantiers" : "886",
    "Porchefontaine" : "679",
    "Montreuil" : "566"
}

stif_code_reverse = {value:key for key, value in stif_code.items()}

In [16]:
test = pd.read_csv('ef_versailles.csv')

In [17]:
test.head()

,Station_name,Network,Longitude,Latitude,City,Traffic_2015,Traffic_2016,Traffic_2017,Traffic_2018,Traffic_2019,Traffic_2020,Traffic_2021,Postcode,Traffic_2022,dept,Traffic_pre_cov,Traffic_pre_cov_sd,covid_drop_2020,covid_drop_2021,Traffic_2020_2021,Origin_id,Rank_rer_2015,Rank_rer_2016,Rank_rer_2017,Rank_rer_2018,Rank_rer_2019,Rank_rer_2020,Rank_rer_2021,Rank_rer_2022
0,Versailles Chantiers,RER,2.135160,48.795690,versailles,24191310.0,24132107.0,23974181.0,23336771.0,22909416.0,12376593,15922909,78000.0,18934500.0,78.0,23708757.0,501966.560353,-0.477974,-0.328395,0.286534,531,8.0,9.0,9.0,10.0,10.0,8.0,11.0,8.0
1,Versailles Château Rive Gauche,RER,2.128880,48.800330,versailles,8748796.0,8367046.0,8009738.0,7520468.0,7015859.0,2886249,3310046,78000.0,5795124.0,78.0,7932381.4,611741.140672,-0.636143,-0.582717,0.146833,457,41.0,49.0,55.0,59.0,64.0,71.0,103.0,62.0
2,Versailles Rive Droite,RER,2.135087,48.809482,versailles,5357929.0,5327810.0,5562438.0,5705373.0,5855418.0,2039014,4698156,78000.0,5530365.0,78.0,5561793.6,201563.810161,-0.633389,-0.155280,1.304131,458,83.0,85.0,81.0,81.0,79.0,109.0,71.0,64.0
3,Montreuil (transilien),RER,2.152993,48.805941,versailles,2248554.0,2137642.0,2192249.0,2204405.0,2226698.0,748793,1897965,78000.0,2010237.0,78.0,2201909.6,37470.103948,-0.659935,-0.138037,1.534699,423,183.0,198.0,196.0,196.0,194.0,231.0,178.0,156.0
4,Porchefontaine,RER,2.152379,48.796528,versailles,956155.0,946641.0,933001.0,902371.0,875987.0,398498,545751,78000.0,723565.0,78.0,922831.0,29643.906328,-0.568179,-0.408612,0.369520,663,264.0,267.0,273.0,275.0,279.0,276.0,301.0,241.0


In [3]:
traffic = pd.read_csv('ef_versailles.csv')
traffic = traffic.drop(["Network", "Longitude", 'Latitude', "City", "Postcode", "dept"], axis = 1)
traffic = traffic.rename(columns = {excol:str(excol).lower() for excol in traffic.columns})
traffic[["traffic_2015","traffic_2016","traffic_2017","traffic_2018","traffic_2019","traffic_2020","traffic_2021","traffic_2022","traffic_pre_cov","traffic_pre_cov_sd"]] = traffic[["traffic_2015","traffic_2016","traffic_2017","traffic_2018","traffic_2019","traffic_2020","traffic_2021","traffic_2022","traffic_pre_cov","traffic_pre_cov_sd"]].astype('int')
traffic[["rank_rer_2015","rank_rer_2016","rank_rer_2017","rank_rer_2018","rank_rer_2019","rank_rer_2020","rank_rer_2021","rank_rer_2022"]] = traffic[["rank_rer_2015","rank_rer_2016","rank_rer_2017","rank_rer_2018","rank_rer_2019","rank_rer_2020","rank_rer_2021","rank_rer_2022"]].astype('int')
traffic.loc[traffic["origin_id"] == 531, "station_name"] = "Versailles Chantiers"
traffic.loc[traffic["origin_id"] == 457, "station_name"] = "Versailles Château Rive Gauche"
traffic.loc[traffic["origin_id"] == 458, "station_name"] = "Versailles Rive Droite"
traffic.loc[traffic["origin_id"] == 423, "station_name"] = "Montreuil"
traffic.loc[traffic["origin_id"] == 663, "station_name"] = "Porchefontaine"
traffic['station_id'] = traffic['station_name'].map(stif_code)
traffic = traffic[["station_name", "station_id", "origin_id", "traffic_2015","traffic_2016","traffic_2017","traffic_2018","traffic_2019","traffic_2020","traffic_2021","traffic_2022","traffic_pre_cov","traffic_pre_cov_sd", "covid_drop_2020","covid_drop_2021","traffic_2020_2021", "rank_rer_2015","rank_rer_2016","rank_rer_2017","rank_rer_2018","rank_rer_2019","rank_rer_2020","rank_rer_2021","rank_rer_2022"]]
traffic.head()

,station_name,station_id,origin_id,traffic_2015,traffic_2016,traffic_2017,traffic_2018,traffic_2019,traffic_2020,traffic_2021,traffic_2022,traffic_pre_cov,traffic_pre_cov_sd,covid_drop_2020,covid_drop_2021,traffic_2020_2021,rank_rer_2015,rank_rer_2016,rank_rer_2017,rank_rer_2018,rank_rer_2019,rank_rer_2020,rank_rer_2021,rank_rer_2022
0,Versailles Chantiers,886,531,24191310,24132107,23974181,23336771,22909416,12376593,15922909,18934500,23708757,501966,-0.477974,-0.328395,0.286534,8,9,9,10,10,8,11,8
1,Versailles Château Rive Gauche,885,457,8748796,8367046,8009738,7520468,7015859,2886249,3310046,5795124,7932381,611741,-0.636143,-0.582717,0.146833,41,49,55,59,64,71,103,62
2,Versailles Rive Droite,884,458,5357929,5327810,5562438,5705373,5855418,2039014,4698156,5530365,5561793,201563,-0.633389,-0.155280,1.304131,83,85,81,81,79,109,71,64
3,Montreuil,566,423,2248554,2137642,2192249,2204405,2226698,748793,1897965,2010237,2201909,37470,-0.659935,-0.138037,1.534699,183,198,196,196,194,231,178,156
4,Porchefontaine,679,663,956155,946641,933001,902371,875987,398498,545751,723565,922831,29643,-0.568179,-0.408612,0.369520,264,267,273,275,279,276,301,241


In [4]:
# 2015

df15_s1 = pd.read_csv('original_datasets/data-rf-2015/2015S1_NB_FER.csv', sep = ";")
df15_s1 = df15_s1[df15_s1['CODE_STIF_ARRET'].isin(list(stif_code.values()))]

df15_s2 = pd.read_csv('original_datasets/data-rf-2015/2015S2_NB_FER.csv', sep = ";")
df15_s2 = df15_s2[df15_s2['CODE_STIF_ARRET'].isin(list(stif_code.values()))]

# 2016

df16_s1 = pd.read_csv('original_datasets/data-rf-2016/2016S1_NB_FER.txt', sep = "\t")
df16_s1 = df16_s1[df16_s1['CODE_STIF_ARRET'].isin(list(stif_code.values()))]

df16_s2 = pd.read_csv('original_datasets/data-rf-2016/2016S2_NB_FER.txt', sep = "\t")
df16_s2 = df16_s2[df16_s2['CODE_STIF_ARRET'].isin(list(stif_code.values()))]

# 2017

df17_s1 = pd.read_csv('original_datasets/data-rf-2017/2017S1_NB_FER.txt', sep = "\t")
df17_s1 = df17_s1[df17_s1['CODE_STIF_ARRET'].isin(list(stif_code.values()))]

df17_s2 = pd.read_csv('original_datasets/data-rf-2017/2017_S2_NB_FER.txt', sep = "\t")
df17_s2 = df17_s2[df17_s2['CODE_STIF_ARRET'].isin(list(stif_code.values()))]

# 2018

df18_s1 = pd.read_csv('original_datasets/data-rf-2018/2018_S1_NB_FER.txt', sep = "\t")
df18_s1 = df18_s1[df18_s1['CODE_STIF_ARRET'].isin(list(stif_code.values()))]

df18_s2 = pd.read_csv('original_datasets/data-rf-2018/2018_S2_NB_FER.txt', sep = "\t")
df18_s2 = df18_s2[df18_s2['CODE_STIF_ARRET'].isin(list(stif_code.values()))]

# 2019

df19_s1 = pd.read_csv('original_datasets/data-rf-2019/2019_S1_NB_FER.txt', sep = "\t")
df19_s1 = df19_s1[df19_s1['CODE_STIF_ARRET'].isin(list(stif_code.values()))]

df19_s2 = pd.read_csv('original_datasets/data-rf-2019/2019_S2_NB_FER.txt', sep = "\t")
df19_s2 = df19_s2[df19_s2['CODE_STIF_ARRET'].isin(list(stif_code.values()))]

# 2020

df20_s1 = pd.read_csv('original_datasets/data-rf-2020/2020_S1_NB_FER.txt', sep = "\t")
df20_s1 = df20_s1[df20_s1['CODE_STIF_ARRET'].isin(list(stif_code.values()))]

df20_s2 = pd.read_csv('original_datasets/data-rf-2020/2020_S2_NB_FER.txt', sep = "\t")
df20_s2 = df20_s2[df20_s2['CODE_STIF_ARRET'].isin(list(stif_code.values()))]

# 2021

df21_s1 = pd.read_csv('original_datasets/data-rf-2021/2021_S1_NB_FER.txt', sep = "\t")
df21_s1 = df21_s1[df21_s1['CODE_STIF_ARRET'].isin(list(stif_code.values()))]

df21_s2 = pd.read_csv('original_datasets/data-rf-2021/2021_S2_NB_FER.txt', sep = "\t")
df21_s2 = df21_s2[df21_s2['CODE_STIF_ARRET'].isin(list(stif_code.values()))]

# 2022

df22_s1 = pd.read_csv('original_datasets/data-rf-2022/2022_S1_NB_FER.txt', sep = "\t")
df22_s1 = df22_s1[df22_s1['CODE_STIF_ARRET'].isin(list(stif_code.values()))]

df22_s2 = pd.read_csv('original_datasets/data-rf-2022/2022_S2_NB_FER.txt', sep = ";")
df22_s2 = df22_s2[df22_s2['CODE_STIF_ARRET'].isin(list(stif_code.values()))]

# 2023

df23_s1 = pd.read_csv('original_datasets/data-rf-2023/2023_S1_NB_FER.csv', sep = ";")
df23_s1 = df23_s1[df23_s1['CODE_STIF_ARRET'].isin(list(stif_code.values()))]

df2 = pd.concat([df15_s1, df15_s2, df16_s1, df16_s2, df17_s1, df17_s2, df18_s1, df18_s2,
                df19_s1, df19_s2, df20_s1, df20_s2, df21_s1, df21_s2, df22_s1, df22_s2,
               ]).reset_index(drop = True)

df2['JOUR'] = pd.to_datetime(df2['JOUR'], format = '%d/%m/%Y')
df23_s1['JOUR'] = pd.to_datetime(df23_s1['JOUR'], format = '%Y-%m-%d')



In [5]:
daily = pd.concat([df2, df23_s1]).reset_index(drop = True)
daily = daily.drop(["CODE_STIF_TRNS", "CODE_STIF_RES", "ID_REFA_LDA", "lda"], axis = 1)
daily = daily.rename(columns = {"JOUR" : "date", "CODE_STIF_ARRET" : "station_id", "LIBELLE_ARRET" : "station_name", "CATEGORIE_TITRE" : "pass_type", "NB_VALD" : "n_passengers"})
daily['station_name'] = daily['station_id'].map(stif_code_reverse)
daily['n_passengers'] = daily.apply(lambda row: 0 if row['n_passengers'] == "Moins de 5" else row['n_passengers'], axis = 1)
daily['n_passengers'] = daily['n_passengers'].astype("int")
daily['year'] = daily['date'].dt.year
daily['day'] = daily['date'].dt.day
daily['month'] = daily['date'].dt.month
daily['day_name'] = daily['date'].dt.day_name()
daily['month_name'] = daily['date'].dt.month_name()

daily.head(50)

,date,station_id,station_name,pass_type,n_passengers,year,day,month,day_name,month_name
0,2015-01-01,679,Porchefontaine,AMETHYSTE,7,2015,1,1,Thursday,January
1,2015-01-01,679,Porchefontaine,AUTRE TITRE,0,2015,1,1,Thursday,January
2,2015-01-01,679,Porchefontaine,FGT,0,2015,1,1,Thursday,January
3,2015-01-01,679,Porchefontaine,IMAGINE R,33,2015,1,1,Thursday,January
4,2015-01-01,679,Porchefontaine,NAVIGO,52,2015,1,1,Thursday,January
5,2015-01-01,679,Porchefontaine,TST,0,2015,1,1,Thursday,January
6,2015-01-01,885,Versailles Château Rive Gauche,AMETHYSTE,28,2015,1,1,Thursday,January
7,2015-01-01,885,Versailles Château Rive Gauche,AUTRE TITRE,21,2015,1,1,Thursday,January
8,2015-01-01,885,Versailles Château Rive Gauche,FGT,35,2015,1,1,Thursday,January
9,2015-01-01,885,Versailles Château Rive Gauche,IMAGINE R,133,2015,1,1,Thursday,January


In [6]:
gares = pd.read_csv('original_datasets/referentiel-gares-voyageurs.csv', sep = ";")
gares["Commune"] = gares["Commune"].dropna()
infos_v = gares[gares["Commune"] == 'Versailles'][["Longitude", 'Latitude', "Intitulé gare"]]
infos_v['stif_code'] = infos_v['Intitulé gare'].map(stif_code)
infos_v = infos_v.rename(columns = {"Longitude" : "longitude", "Latitude" : "latitude", "Intitulé gare" : "station_name", "stif_code" : "station_id"})
infos_v = infos_v.reset_index(drop = True)
infos_v = infos_v[['station_name', 'station_id', 'latitude', 'longitude']]

In [7]:
infos_v.head()

,station_name,station_id,latitude,longitude
0,Montreuil,566,48.806732,2.151169
1,Versailles Rive Droite,884,48.809652,2.134752
2,Porchefontaine,679,48.796938,2.154016
3,Versailles Château Rive Gauche,885,48.800364,2.128827
4,Versailles Chantiers,886,48.795826,2.135883


In [8]:
count_code = {
    "Versailles Rive Droite" : 87382861,
    "Versailles Château Rive Gauche" : 87393157,
    "Versailles Chantiers" : 87393009,
    "Porchefontaine" : 87393165,
    "Montreuil" : 87382879
}

In [9]:
count = pd.read_csv('original_datasets/comptage-transilien.csv', sep = ";")
count = count[count['Code Gare'].isin(count_code.values())]
count = count.rename(columns = {"Nom gare" : "station_name", "Code Gare" : "station_id", "Type jour" : "day_type", "Année" : "year", "Date de comptage" : "date", "Ligne" : "line", "Tranche horaire" : "time_period", "Montants" : "counts"})
count = count.drop("Axe", axis = 1)
count['station_name'] = count['station_id'].map({value:key for key,value in count_code.items()})
count['station_id'] = count['station_name'].map(stif_code)
count['day_type'] = count['day_type'].map({"JOB":'weekday', "Samedi": "saturday", "Dimanche":"sunday"})
count['time_period'] = count['time_period'].map({"Avant 6h" : 6, "De 6h à 10h": 10, "De 10h à 16h": 16, "De 16h à 20h":20 ,"Après 20h" : 23})
count['date'] = pd.to_datetime(count['date'], format = '%Y-%m-%d')
count.head()

,station_name,station_id,day_type,year,date,line,time_period,counts
218,Porchefontaine,679,sunday,2017,2017-04-23,C,23,46
349,Porchefontaine,679,weekday,2017,2017-04-25,C,16,337
359,Porchefontaine,679,saturday,2017,2017-04-22,C,23,63
375,Versailles Chantiers,886,sunday,2017,2017-04-23,C,16,724
586,Porchefontaine,679,saturday,2017,2017-04-22,C,6,7


In [10]:
traffic.to_csv('ef_v_traffic.csv', index = False)
daily.to_csv('ef_v_daily.csv', index = False)
infos_v.to_csv('ef_v_infos.csv', index = False)
count.to_csv('ef_v_count.csv', index = False)

In [11]:
traffic.head()

,station_name,station_id,origin_id,traffic_2015,traffic_2016,traffic_2017,traffic_2018,traffic_2019,traffic_2020,traffic_2021,traffic_2022,traffic_pre_cov,traffic_pre_cov_sd,covid_drop_2020,covid_drop_2021,traffic_2020_2021,rank_rer_2015,rank_rer_2016,rank_rer_2017,rank_rer_2018,rank_rer_2019,rank_rer_2020,rank_rer_2021,rank_rer_2022
0,Versailles Chantiers,886,531,24191310,24132107,23974181,23336771,22909416,12376593,15922909,18934500,23708757,501966,-0.477974,-0.328395,0.286534,8,9,9,10,10,8,11,8
1,Versailles Château Rive Gauche,885,457,8748796,8367046,8009738,7520468,7015859,2886249,3310046,5795124,7932381,611741,-0.636143,-0.582717,0.146833,41,49,55,59,64,71,103,62
2,Versailles Rive Droite,884,458,5357929,5327810,5562438,5705373,5855418,2039014,4698156,5530365,5561793,201563,-0.633389,-0.155280,1.304131,83,85,81,81,79,109,71,64
3,Montreuil,566,423,2248554,2137642,2192249,2204405,2226698,748793,1897965,2010237,2201909,37470,-0.659935,-0.138037,1.534699,183,198,196,196,194,231,178,156
4,Porchefontaine,679,663,956155,946641,933001,902371,875987,398498,545751,723565,922831,29643,-0.568179,-0.408612,0.369520,264,267,273,275,279,276,301,241


In [15]:
daily.tail()

,date,station_id,station_name,pass_type,n_passengers,year,day,month,day_name,month_name
103952,2023-06-26,884,Versailles Rive Droite,FGT,114,2023,26,6,Monday,June
103953,2023-06-26,884,Versailles Rive Droite,IMAGINE R,1247,2023,26,6,Monday,June
103954,2023-06-26,884,Versailles Rive Droite,NAVIGO,3433,2023,26,6,Monday,June
103955,2023-06-26,884,Versailles Rive Droite,NAVIGO JOUR,26,2023,26,6,Monday,June
103956,2023-06-26,884,Versailles Rive Droite,NON DEFINI,207,2023,26,6,Monday,June


In [13]:
infos_v.head()

,station_name,station_id,latitude,longitude
0,Montreuil,566,48.806732,2.151169
1,Versailles Rive Droite,884,48.809652,2.134752
2,Porchefontaine,679,48.796938,2.154016
3,Versailles Château Rive Gauche,885,48.800364,2.128827
4,Versailles Chantiers,886,48.795826,2.135883


In [14]:
count.head()

,station_name,station_id,day_type,year,date,line,time_period,counts
218,Porchefontaine,679,sunday,2017,2017-04-23,C,23,46
349,Porchefontaine,679,weekday,2017,2017-04-25,C,16,337
359,Porchefontaine,679,saturday,2017,2017-04-22,C,23,63
375,Versailles Chantiers,886,sunday,2017,2017-04-23,C,16,724
586,Porchefontaine,679,saturday,2017,2017-04-22,C,6,7
